In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Implementacja sieci neuronowej od podstaw
class NeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size, activation='sigmoid', learning_rate=0.01, problem_type='regression'):
        self.layers = [input_size] + hidden_layers + [output_size]
        self.activation = activation
        self.learning_rate = learning_rate
        self.problem_type = problem_type
        self.weights = []
        self.biases = []

        # Inicjalizacja wag i bias
        for i in range(len(self.layers) - 1):
            w = np.random.randn(self.layers[i], self.layers[i+1]) * 0.5
            b = np.zeros((1, self.layers[i+1]))
            self.weights.append(w)
            self.biases.append(b)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def tanh(self, x):
        return np.tanh(x)

    def tanh_derivative(self, x):
        return 1 - x**2

    def leaky_relu(self, x, alpha=0.01):
        return np.where(x > 0, x, alpha * x)

    def leaky_relu_derivative(self, x, alpha=0.01):
        return np.where(x > 0, 1, alpha)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def activate(self, x):
        if self.activation == 'sigmoid':
            return self.sigmoid(x)
        elif self.activation == 'relu':
            return self.relu(x)
        elif self.activation == 'tanh':
            return self.tanh(x)
        elif self.activation == 'leaky_relu':
            return self.leaky_relu(x)

    def activate_derivative(self, z):
        if self.activation == 'sigmoid':
            sig = self.sigmoid(z)
            return sig * (1 - sig)
        elif self.activation == 'relu':
            return self.relu_derivative(z)
        elif self.activation == 'tanh':
            return 1 - np.tanh(z)**2
        elif self.activation == 'leaky_relu':
            return self.leaky_relu_derivative(z)

    def forward(self, X):
        self.z_values = []  # pre-activation values
        self.activations = [X]

        for i in range(len(self.weights)):
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.z_values.append(z)

            if i == len(self.weights) - 1:  # ostatnia warstwa
                if self.problem_type == 'classification' and self.layers[-1] > 1:
                    a = self.softmax(z)
                elif self.problem_type == 'classification':
                    a = self.sigmoid(z)
                else:
                    a = z
            else:
                a = self.activate(z)
            self.activations.append(a)

        return self.activations[-1]

    def backward(self, X, y, output):
        m = X.shape[0]

        # Błąd dla ostatniej warstwy
        if self.problem_type == 'classification' and self.layers[-1] > 1:
            # One-hot encoding dla y
            y_onehot = np.zeros((m, self.layers[-1]))
            y_onehot[np.arange(m), y.astype(int)] = 1
            delta = output - y_onehot
        else:
            delta = output - y.reshape(-1, 1) if len(y.shape) == 1 else output - y
        for i in reversed(range(len(self.weights))):
            self.weights[i] -= self.learning_rate * np.dot(self.activations[i].T, delta) / m
            self.biases[i] -= self.learning_rate * np.sum(delta, axis=0, keepdims=True) / m

            if i > 0:
                delta = np.dot(delta, self.weights[i].T) * self.activate_derivative(self.z_values[i-1])

    def train(self, X, y, epochs=1000, verbose=False):
        losses = []
        accuracies = []

        for epoch in range(epochs):
            output = self.forward(X)

            # Obliczenie loss
            if self.problem_type == 'classification' and self.layers[-1] > 1:
                # Cross-entropy loss dla klasyfikacji wieloklasowej
                y_onehot = np.zeros((len(y), self.layers[-1]))
                y_onehot[np.arange(len(y)), y.astype(int)] = 1
                loss = -np.mean(np.sum(y_onehot * np.log(output + 1e-15), axis=1))

                # Obliczenie accuracy dla klasyfikacji
                predictions = np.argmax(output, axis=1)
                accuracy = np.mean(predictions == y)
                accuracies.append(accuracy)
            else:
                # MSE loss
                if len(y.shape) == 1:
                    y = y.reshape(-1, 1)
                loss = np.mean((output - y)**2)
                accuracies.append(0)  # Dla regresji nie obliczamy accuracy

            losses.append(loss)
            self.backward(X, y, output)

            if verbose and epoch % 100 == 0:
                if self.problem_type == 'classification':
                    print(f'Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')
                else:
                    print(f'Epoch {epoch}, Loss: {loss:.4f}')

        # Zapisanie historii treningu
        self.history = {'loss': losses}
        if self.problem_type == 'classification':
            self.history['accuracy'] = accuracies

        return losses

    def predict(self, X):
        output = self.forward(X)
        if self.problem_type == 'classification' and self.layers[-1] > 1:
            return np.argmax(output, axis=1)
        else:
            return output

# Funkcja do przygotowania danych
def prepare_data():
    print("Ładowanie i przygotowanie danych...")

    # Wczytanie danych
    df = pd.read_csv('train_data.csv', sep=';')

    print(f"Rozmiar danych: {df.shape}")
    print(f"Kolumny: {df.columns.tolist()}")

    # Usunięcie niepotrzebnych kolumn
    df = df.drop(['case_id', 'patientid'], axis=1)

    # Kodowanie zmiennych kategorycznych
    le_dict = {}
    categorical_columns = ['Hospital_type_code', 'Department', 'Ward_Type', 'Ward_Facility_Code',
                          'Type of Admission', 'Severity of Illness', 'Age', 'Stay','Hospital_region_code']

    for col in categorical_columns:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            le_dict[col] = le

    # Uzupełnienie brakujących wartości
    df = df.fillna(df.mean())

    print(f"Liczba kolumn po przetworzeniu: {df.shape[1]}")
    print(f"Kolumny finalne: {df.columns.tolist()}")

    return df, le_dict

# Funkcja do wizualizacji wyników dla najlepszego modelu
def visualize_best_model(X_train, X_test, y_train, y_test, best_params, problem_type='classification'):
    """Trenuje najlepszy model i wizualizuje wyniki"""

    print(f"\nTrenowanie najlepszego modelu z parametrami: {best_params}")

    # Utworzenie najlepszego modelu
    nn = NeuralNetwork(
        input_size=X_train.shape[1],
        hidden_layers=best_params['hidden_layers'],
        output_size=5 if problem_type == 'classification' else 1,
        activation=best_params['activation'],
        learning_rate=best_params['learning_rate'],
        problem_type=problem_type
    )

    # Trenowanie z verbose=True żeby widzieć postęp
    losses = nn.train(X_train, y_train, epochs=best_params['epochs'], verbose=True)

    # Predykcje
    y_pred_test = nn.predict(X_test)
    y_pred_train = nn.predict(X_train)

    if problem_type == 'classification':
        # Nazwy klas
        class_names = ['0-10 dni', '11-20 dni', '21-30 dni', '31-40 dni', '>40 dni']

        # Accuracy
        train_acc = accuracy_score(y_train, y_pred_train)
        test_acc = accuracy_score(y_test, y_pred_test)

        print(f"\nWyniki najlepszego modelu:")
        print(f"Accuracy na zbiorze treningowym: {train_acc:.4f}")
        print(f"Accuracy na zbiorze testowym: {test_acc:.4f}")

        # Macierz pomyłek
        cm = confusion_matrix(y_test, y_pred_test)

        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names, yticklabels=class_names)
        plt.title('Macierz pomyłek - Najlepszy model')
        plt.xlabel('Przewidywane')
        plt.ylabel('Rzeczywiste')
        plt.show()

        # Raport klasyfikacji
        print("\nRaport klasyfikacji:")
        print(classification_report(y_test, y_pred_test, target_names=class_names))

        # Wykres historii uczenia
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(nn.history['loss'])
        plt.title('Loss podczas treningu')
        plt.xlabel('Epoka')
        plt.ylabel('Loss')
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(nn.history['accuracy'])
        plt.title('Accuracy podczas treningu')
        plt.xlabel('Epoka')
        plt.ylabel('Accuracy')
        plt.grid(True)

        plt.tight_layout()
        plt.show()

        return nn, test_acc

    else:  # regression
        y_pred_test = y_pred_test.flatten()
        y_pred_train = y_pred_train.flatten()

        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        print(f"\nWyniki najlepszego modelu:")
        print(f"RMSE na zbiorze treningowym: {train_rmse:.4f}")
        print(f"RMSE na zbiorze testowym: {test_rmse:.4f}")

        # Wykres predykcji vs rzeczywiste wartości
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.scatter(y_test, y_pred_test, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        plt.xlabel('Rzeczywiste wartości')
        plt.ylabel('Przewidywane wartości')
        plt.title('Predykcje vs Rzeczywiste wartości (Test)')
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(nn.history['loss'])
        plt.title('Loss podczas treningu')
        plt.xlabel('Epoka')
        plt.ylabel('Loss (MSE)')
        plt.grid(True)

        plt.tight_layout()
        plt.show()

        return nn, test_rmse
def test_parameters(X_train, X_test, y_train, y_test, param_name, param_values, base_params, problem_type='regression'):
    results = []

    print(f"\nTestowanie parametru: {param_name}")

    for param_value in param_values:
        print(f"  Testowanie {param_name} = {param_value}")

        # Tworzenie parametrów dla tej iteracji
        current_params = base_params.copy()
        current_params[param_name] = param_value

        # Wykonanie kilku prób dla każdego zestawu parametrów
        train_scores = []
        test_scores = []

        for trial in range(3):  # 3 próby dla każdego zestawu
            if param_name == 'hidden_layers':
                nn = NeuralNetwork(
                    input_size=X_train.shape[1],
                    hidden_layers=param_value,
                    output_size=5 if problem_type == 'classification' else 1,  # 5 klas dla klasyfikacji
                    activation=current_params.get('activation', 'sigmoid'),
                    learning_rate=current_params.get('learning_rate', 0.01),
                    problem_type=problem_type
                )
            else:
                output_size = 5 if problem_type == 'classification' else 1
                nn = NeuralNetwork(
                    input_size=X_train.shape[1],
                    hidden_layers=current_params.get('hidden_layers', [10]),
                    output_size=output_size,
                    activation=current_params.get('activation', 'sigmoid') if param_name != 'activation' else param_value,
                    learning_rate=current_params.get('learning_rate', 0.01) if param_name != 'learning_rate' else param_value,
                    problem_type=problem_type
                )

            epochs = current_params.get('epochs', 500) if param_name != 'epochs' else param_value
            nn.train(X_train, y_train, epochs=epochs)

            # Predykcje
            train_pred = nn.predict(X_train)
            test_pred = nn.predict(X_test)

            # Obliczenie metryk
            if problem_type == 'regression':
                train_pred = train_pred.flatten()
                test_pred = test_pred.flatten()
                train_score = np.sqrt(mean_squared_error(y_train, train_pred))
                test_score = np.sqrt(mean_squared_error(y_test, test_pred))
            else:  # classification
                train_score = accuracy_score(y_train, train_pred)
                test_score = accuracy_score(y_test, test_pred)

            train_scores.append(train_score)
            test_scores.append(test_score)

        # Zapisanie wyników
        results.append({
            'parameter': param_name,
            'value': param_value,
            'train_mean': np.mean(train_scores),
            'train_best': np.min(train_scores) if problem_type == 'regression' else np.max(train_scores),
            'test_mean': np.mean(test_scores),
            'test_best': np.min(test_scores) if problem_type == 'regression' else np.max(test_scores)
        })

    return results

# Główna funkcja
def main():
    print("=== PROJEKT SZTUCZNE SIECI NEURONOWE ===")
    print("Analiza danych szpitalnych")

    # Przygotowanie danych
    df, le_dict = prepare_data()

    # ========================================
    # PROBLEM REGRESYJNY - przewidywanie Admission_Deposit
    # ========================================
    print("\n" + "="*50)
    print("PROBLEM REGRESYJNY - Przewidywanie Admission_Deposit")
    print("="*50)

    # Przygotowanie danych dla regresji
    X_reg = df.drop(['Admission_Deposit'], axis=1).values
    y_reg = df['Admission_Deposit'].values

    print(f"Liczba cech dla problemu regresyjnego: {X_reg.shape[1]}")
    print(f"Liczba próbek: {X_reg.shape[0]}")

    # Normalizacja danych
    scaler_X_reg = StandardScaler()
    scaler_y_reg = StandardScaler()

    X_reg_scaled = scaler_X_reg.fit_transform(X_reg)
    y_reg_scaled = scaler_y_reg.fit_transform(y_reg.reshape(-1, 1)).flatten()

    # Podział na zbiory
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
        X_reg_scaled, y_reg_scaled, test_size=0.2, random_state=42
    )

    # Parametry bazowe dla regresji
    base_params_reg = {
        'hidden_layers': [10],
        'activation': 'sigmoid',
        'learning_rate': 0.01,
        'epochs': 500
    }

    # Testowanie parametrów dla regresji
    all_results_reg = []

    # 1. Liczba neuronów w warstwie ukrytej
    neuron_counts = [[5], [10], [20], [30]]
    results = test_parameters(X_train_reg, X_test_reg, y_train_reg, y_test_reg,
                            'hidden_layers', neuron_counts, base_params_reg, 'regression')
    all_results_reg.extend(results)

    # 2. Funkcja aktywacji - zmienione na 4 działające funkcje
    activations = ['sigmoid', 'relu', 'tanh', 'leaky_relu']
    results = test_parameters(X_train_reg, X_test_reg, y_train_reg, y_test_reg,
                            'activation', activations, base_params_reg, 'regression')
    all_results_reg.extend(results)

    # 3. Współczynnik uczenia
    learning_rates = [0.001, 0.01, 0.1, 0.2]
    results = test_parameters(X_train_reg, X_test_reg, y_train_reg, y_test_reg,
                            'learning_rate', learning_rates, base_params_reg, 'regression')
    all_results_reg.extend(results)

    # ========================================
    # PROBLEM KLASYFIKACYJNY - klasyfikacja Stay
    # ========================================
    print("\n" + "="*50)
    print("PROBLEM KLASYFIKACYJNY - Klasyfikacja długości pobytu")
    print("="*50)

    # Przygotowanie danych dla klasyfikacji - kategorie długości pobytu
    # Tworzenie kategorii: wszystko >40 dni w jednej kategorii
    def categorize_stay(stay_encoded):
        # Odwrócenie kodowania aby zobaczyć oryginalne wartości
        stay_original = le_dict['Stay'].inverse_transform([stay_encoded])[0]

        # Mapowanie kategorii
        if '0-10' in stay_original:
            return 0  # krótki pobyt
        elif '11-20' in stay_original:
            return 1  # średni pobyt
        elif '21-30' in stay_original:
            return 2  # długi pobyt
        elif '31-40' in stay_original:
            return 3  # bardzo długi pobyt
        else:  # wszystko powyżej 40 dni
            return 4  # najdłuższy pobyt

    # Przygotowanie danych
    X_class = df.drop(['Stay'], axis=1).values
    y_class_original = df['Stay'].values

    # Utworzenie nowych kategorii
    y_class = np.array([categorize_stay(stay) for stay in y_class_original])

    print(f"Liczba cech dla problemu klasyfikacyjnego: {X_class.shape[1]}")
    print(f"Liczba próbek: {X_class.shape[0]}")
    print(f"Rozkład klas: {np.bincount(y_class)}")
    print("Kategorie: 0=0-10dni, 1=11-20dni, 2=21-30dni, 3=31-40dni, 4=>40dni")

    # Normalizacja danych
    scaler_X_class = StandardScaler()
    X_class_scaled = scaler_X_class.fit_transform(X_class)

    # Podział na zbiory
    X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
        X_class_scaled, y_class, test_size=0.2, random_state=42
    )
    base_params_class = {
        'hidden_layers': [20, 10],
        'activation': 'relu',
        'learning_rate': 0.005,
        'epochs': 1000
    }

    # Testowanie parametrów dla klasyfikacji
    all_results_class = []

    # 1. Liczba neuronów w warstwie ukrytej
    results = test_parameters(X_train_class, X_test_class, y_train_class, y_test_class,
                            'hidden_layers', neuron_counts, base_params_class, 'classification')
    all_results_class.extend(results)

    # 2. Funkcja aktywacji - zmienione na 4 działające funkcje
    results = test_parameters(X_train_class, X_test_class, y_train_class, y_test_class,
                            'activation', activations, base_params_class, 'classification')
    all_results_class.extend(results)

    # 3. Współczynnik uczenia
    results = test_parameters(X_train_class, X_test_class, y_train_class, y_test_class,
                            'learning_rate', learning_rates, base_params_class, 'classification')
    all_results_class.extend(results)

    # ========================================
    # WYNIKI
    # ========================================
    print("\n" + "="*50)
    print("WYNIKI ANALIZY")
    print("="*50)

    print("\nPROBLEM REGRESYJNY (RMSE - im mniej, tym lepiej):")
    print("-" * 60)
    print(f"{'Parametr':<15} {'Wartość':<15} {'Train śr.':<12} {'Train najl.':<12} {'Test śr.':<12} {'Test najl.':<12}")
    print("-" * 60)
    for result in all_results_reg:
        print(f"{result['parameter']:<15} {str(result['value']):<15} {result['train_mean']:.4f}      {result['train_best']:.4f}       {result['test_mean']:.4f}     {result['test_best']:.4f}")

    print("\nPROBLEM KLASYFIKACYJNY (Accuracy - im więcej, tym lepiej):")
    print("-" * 60)
    print(f"{'Parametr':<15} {'Wartość':<15} {'Train śr.':<12} {'Train najl.':<12} {'Test śr.':<12} {'Test najl.':<12}")
    print("-" * 60)
    for result in all_results_class:
        print(f"{result['parameter']:<15} {str(result['value']):<15} {result['train_mean']:.4f}      {result['train_best']:.4f}       {result['test_mean']:.4f}     {result['test_best']:.4f}")

    # ========================================
    # WNIOSKI
    # ========================================
    print("\n" + "="*50)
    print("WNIOSKI")
    print("="*50)

    print("\nPROBLEM REGRESYJNY:")
    best_reg = min(all_results_reg, key=lambda x: x['test_best'])
    print(f"- Najlepszy wynik: {best_reg['parameter']} = {best_reg['value']} (RMSE: {best_reg['test_best']:.4f})")

    print("\nPROBLEM KLASYFIKACYJNY:")
    best_class = max(all_results_class, key=lambda x: x['test_best'])
    print(f"- Najlepszy wynik: {best_class['parameter']} = {best_class['value']} (Accuracy: {best_class['test_best']:.4f})")

    # ========================================
    # WIZUALIZACJA NAJLEPSZYCH MODELI
    # ========================================
    print("\n" + "="*50)
    print("WIZUALIZACJA NAJLEPSZYCH MODELI")
    print("="*50)

    # Najlepsze parametry dla regresji
    best_reg_params = {
        'hidden_layers': [10],
        'activation': 'sigmoid',
        'learning_rate': 0.01,
        'epochs': 500
    }

    # Znajdź najlepsze parametry z wyników
    for result in all_results_reg:
        if result['test_best'] == best_reg['test_best']:
            if result['parameter'] == 'hidden_layers':
                best_reg_params['hidden_layers'] = result['value']
            elif result['parameter'] == 'activation':
                best_reg_params['activation'] = result['value']
            elif result['parameter'] == 'learning_rate':
                best_reg_params['learning_rate'] = result['value']

    print("\nWizualizacja najlepszego modelu regresyjnego:")
    nn_reg, best_rmse = visualize_best_model(X_train_reg, X_test_reg, y_train_reg, y_test_reg,
                                           best_reg_params, 'regression')

    # Najlepsze parametry dla klasyfikacji
    best_class_params = {
        'hidden_layers': [20, 10],
        'activation': 'relu',
        'learning_rate': 0.005,
        'epochs': 1000
    }

    # Znajdź najlepsze parametry z wyników
    for result in all_results_class:
        if result['test_best'] == best_class['test_best']:
            if result['parameter'] == 'hidden_layers':
                best_class_params['hidden_layers'] = result['value']
            elif result['parameter'] == 'activation':
                best_class_params['activation'] = result['value']
            elif result['parameter'] == 'learning_rate':
                best_class_params['learning_rate'] = result['value']

    print("\nWizualizacja najlepszego modelu klasyfikacyjnego:")
    nn_class, best_acc = visualize_best_model(X_train_class, X_test_class, y_train_class, y_test_class,
                                            best_class_params, 'classification')

    print("\nOGÓLNE WNIOSKI:")
    print("1. Sieć neuronowa została zaimplementowana od podstaw bez gotowych bibliotek")
    print("2. Przeanalizowano wpływ różnych parametrów na skuteczność sieci")
    print("3. Dla każdego parametru przetestowano co najmniej 4 różne wartości")
    print("4. Proces uczenia powtarzano kilkakrotnie dla zwiększenia wiarygodności wyników")
    print("5. Przedstawiono wyniki zarówno dla zbioru treningowego jak i testowego")

if __name__ == "__main__":
    main()

=== PROJEKT SZTUCZNE SIECI NEURONOWE ===
Analiza danych szpitalnych
Ładowanie i przygotowanie danych...
Rozmiar danych: (318438, 18)
Kolumny: ['case_id', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital', 'Hospital_region_code', 'Available Extra Rooms in Hospital', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade', 'patientid', 'City_Code_Patient', 'Type of Admission', 'Severity of Illness', 'Visitors with Patient', 'Age', 'Admission_Deposit', 'Stay']
Liczba kolumn po przetworzeniu: 16
Kolumny finalne: ['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital', 'Hospital_region_code', 'Available Extra Rooms in Hospital', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade', 'City_Code_Patient', 'Type of Admission', 'Severity of Illness', 'Visitors with Patient', 'Age', 'Admission_Deposit', 'Stay']

PROBLEM REGRESYJNY - Przewidywanie Admission_Deposit
Liczba cech dla problemu regresyjnego: 15
Liczba próbek: 318438

Testowanie parametru: hidden_layers
  Tes